# 分散ブロックチェーンの実装

分散ブロックチェーンで各ノードが、ドローンの離着陸地点の予約を管理してるポイントだとした時、ドローンの着陸離陸許可がされる様子をデモする

許可型分散ブロックチェーンにおいて論理的な単一性を実現します

- 私たちの論理的に単一な許可型分散ブロックチェーンにおいては発生したトランザクションは時系列毎に加えられます
    1. 許可型を前提している
        - そのため、参加するノードを特定のネットワーク構造に組み込む事ができる
    2. 分散ブロックチェーンの中で常に特定の１つのブロックチェーンが最長であるようにする
        - ネットワーク形状の指定（１つのリーダーノードを持つ、スター型の分散ネットワーク）
        - その結果として、ブロックが生成された際にはリーダーのみにブロードキャストを行う事となる
        - それは、リーダーノードが常に他よりも長いチェーンを生成する事を可能にする
    3. 複数存在するチェーンを常に最も長いものに同期する
        - 結果、リーダーノードが持つチェーンが常に論理的に単一なチェーンとして選択されます


以上をデモを持って確認する

In [1]:
from __future__ import annotations
from dataclasses import dataclass

In [2]:
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
import binascii

import json
import dataclasses as dc
from typing import List

import multiprocessing as mp
import random
from itertools import count
from time import sleep

from typing import Tuple, Sequence
import hashlib

from time import time

In [3]:
import sys
sys.path.append('src/')

In [4]:
from block import Block
from blockchain import BlockChain
from network import Network
from wallet import Wallet
from smartcontract import Smartcontract

## ノード1つの場合のデモ

In [9]:
genesis=Block(time(), (), "0")
uuid='the_leader_node'
network=Network({uuid:[uuid]})
node=Node(network, genesis, uuid)

In [10]:
alice=Wallet(network, [uuid])
bob=Wallet(network, [uuid])

In [11]:
node.work(verbose=False)

KeyboardInterrupt: 

In [12]:
network.shutdown()

# デモ: 分散ブロックチェーンのデモ

In [13]:
genesis=Block(time(), (), "0")
network=Network()

- ノードを生成する
    - ノードにはデモで分かりやすいように名前を与えてある

In [14]:
nodes=['the_leader_node', 'the_blue_sky', 'white_noise', 'yellow_submarine', 'green_planet']


- nodeのnetworkの状態を定義する
- 序列を持つ結合と全結合を選択できる
    - リーダーノードは他のノードと結合しているが、その他のノードはリーダーとのみ結合している
    - 全結合の場合は全てのノード同士が結合している

（考察）
- 1つのnodeはただ１つの決まった結合を持つ場合、コンフリクトは起こらない
- ブロックが作成されるたびにリーダーにブロードキャストしたい

In [15]:
ORDERING=True
if (ORDERING):
    network.neighbours['the_leader_node']=['the_leader_node', 'the_blue_sky', 'white_noise', 'yellow_submarine', 'green_planet']
    network.neighbours['the_blue_sky']=['the_leader_node', 'the_blue_sky']
    network.neighbours['white_noise']=['the_leader_node', 'white_noise']
    network.neighbours['yellow_submarine']=['the_leader_node', 'yellow_submarine']
    network.neighbours['green_planet']=['the_leader_node', 'green_planet']
else:
    for uuid in nodes:
        network.neighbours[uuid]=nodes

- ノードを起動する

In [16]:
def node_work(uuid):
    node=Node(network, genesis, uuid)
    node.work()

In [17]:
workers=[]
for uuid in nodes:
    worker=mp.Process(target=node_work, args=(uuid,))
    workers.append(worker)
    worker.start()

In [18]:
# num_people=2
# people=[Wallet(network, random.sample(nodes, random.randint(1, len(nodes)))) for _ in range(num_people)]
alice=Wallet(network, ['the_leader_node'])
bob=Wallet(network, ['the_blue_sky'])

people=[alice, bob]

### ブロックチェーンの表示
- 各ノードの持つブロックチェーンがどの様になっているのか表示する

In [ ]:
from IPython.display import clear_output
import time

timeout = time.time() + 60
def draw_chains(network):
    while True:
        if time.time() > timeout:
            break
        line=""
        for k, v in network.chains.items():
            line += f"(Node: {k[:15]}) "
            chain=BlockChain.json_loads(v)
            h=list(map(lambda x: x.hash()[:5], chain))
            line += "--".join(map(lambda x: f"[{x}]", h))
            line += "\n"
        print(line.rstrip())
        clear_output(True)
        sleep(1.0)

worker=mp.Process(target=draw_chains, args=(network,))
worker.start()

(Node: the_leader_node) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]
(Node: the_blue_sky) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]
(Node: white_noise) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]
(Node: yellow_submarin) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]
(Node: green_planet) [a452b]--[24289]--[6a1cb]--[927cc]--[dee79]


In [20]:
def generate_transactions():
    # for _ in range(random.randint(1,10)):
    #     sender, receiver= random.sample(people, 2)
    #     # sender.send(receiver.address, random.randint(1, 100))
    #     sender.send(receiver.address, 'inquiry')

    alice.send([None], 'cancel')
    bob.send([None], 'reservation')

(Node: the_leader_node) [a452b]
(Node: the_blue_sky) [a452b]
(Node: white_noise) [a452b]
(Node: yellow_submarin) [a452b]
(Node: green_planet) [a452b]


In [21]:
# generate_transactions()

### スマートコントラクトの実装に向けて

- 以下のシナリオでデモを実行する

        1. the_leader_nodeが離着陸許可を取得する
        2. the_leader_nodeが離着陸許可をキャンセルする
        3. the_blue_skyが離着陸許可を取得する
        4. the_blue_skyが離着陸許可をキャンセルする 

- ブロックチェーンにおいて以上のトランザクションの履歴を確認する

In [22]:
sc = Smartcontract(network)

s_uuid = 'the_leader_node'
s = Wallet(network, [s_uuid])
b_uuid = 'the_blue_sky'
b = Wallet(network, [b_uuid])

the_leader_node は離着陸地点を予約しました

In [23]:
clear_output(True)

print(s_uuid + ' は離着陸地点を予約しました。' if sc.make_a_reservation(s) else s_uuid + ' は離着陸地点は予約できませんでした。')

the_leader_node は離着陸地点を予約しました。


In [24]:
clear_output(True)

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの現在のステート')
print(sc.get_current_state(s_uuid))


以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': ['the_leader_node'], 'receiver_address': [None], 'reserved': 1}


In [25]:
clear_output(True)

print(s_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(s) else s_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

the_leader_node により離着陸地点の予約はキャンセルされました。


In [26]:
clear_output(True)

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの現在のステート')
print(sc.get_current_state(s_uuid))


以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}
1 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "cancel", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': None, 'receiver_address': None, 'reserved': 0}


In [27]:
clear_output(True)

print(b_uuid + ' は離着陸地点を予約しました。' if sc.make_a_reservation(b) else b_uuid + ' は離着陸地点は予約できませんでした。')

the_blue_sky は離着陸地点を予約しました。


In [28]:
clear_output(True)

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの現在のステート')
print(sc.get_current_state(s_uuid))


以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}
1 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "cancel", "sign": null}
2 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "reservation", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': ['the_blue_sky'], 'receiver_address': [None], 'reserved': 1}


In [29]:
clear_output(True)

print(b_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(b) else b_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

the_blue_sky により離着陸地点の予約はキャンセルされました。


In [31]:
clear_output(True)

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの現在のステート')
print(sc.get_current_state(s_uuid))


以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}
1 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "cancel", "sign": null}
2 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "reservation", "sign": null}
3 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "cancel", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': None, 'receiver_address': None, 'reserved': 0}


In [32]:
clear_output(True)

print(b_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(b) else b_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

print('\n以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('\n以下ブロックチェーンの現在のステート')
print(sc.get_current_state(s_uuid))

the_blue_sky によっては離着陸地点はキャンセルできませんでした。

以下ブロックチェーンの内容（トランザクションの履歴）
0 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "reservation", "sign": null}
1 {"sender_address": ["the_leader_node"], "receiver_address": [null], "value": "cancel", "sign": null}
2 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "reservation", "sign": null}
3 {"sender_address": ["the_blue_sky"], "receiver_address": [null], "value": "cancel", "sign": null}

以下ブロックチェーンの源蔵のステート
{'sender_address': None, 'receiver_address': None, 'reserved': 0}


In [ ]:
worker.terminate()
worker.kill()
for worker in workers:
    worker.terminate()
    worker.kill()
network.shutdown()

以上になります